In [1]:
from misc import *
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau, MultiStepLR

%load_ext autoreload
%autoreload 2
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

# Parameters for loading the .mat file
data_dir = "C:/Users/Mircea/Google Drive/Data/nn-dipole-fitting/"
path_list = [each.replace("\\","/") for each in glob(data_dir + "Dataset SNR*.mat")]
# path_train = data_dir + "Clean Dataset Short.mat"
path_train = data_dir + "Clean Dataset.mat"

# Define parameters
batch_size = 50000
n_epochs = 10

device = check_cuda()

CUDA is available!  Training on GPU ...
Using GeForce RTX 2070


In [2]:
# Read the data if not already in memory
if "dataset" not in globals():
    dataset = Dataset(path_train, path_list[-1])

In [3]:
from misc import dipfit
model = dipfit(dataset.n_chan)
model = model.to(device)

optimizer = optim.Adam(model.parameters(),3e-4) 

In [4]:
model, train_loss, valid_loss = train(model,dataset,n_epochs,batch_size,device,optimizer,"best_model.pt",show_every=1)

8 epochs ready in 857.942 seconds. Minimum validation loss: 1.039565
Train batch time: 0.02437 ± 0.00032 seconds
Valid batch time: 0.02016 ± 0.00035 seconds


In [5]:
# Test dipoles location - ground truth
test_dipoles = torch.from_numpy(dataset.data_test["dipoles"]).to(device)
# Calculate test loss for the network
pred_dipoles_nn = model(torch.from_numpy(dataset.data_test["fields"]).to(device))
loc_error_nn = torch.mean(get_localization_error(pred_dipoles_nn,test_dipoles)).item()
# Test loss for Levenberg-Marquadt
pred_dipoles_lm = torch.from_numpy(dataset.data_test["dipoles_estimated"]).to(device)
loc_error_lm = torch.mean(get_localization_error(pred_dipoles_lm,test_dipoles)).item()
print(f"Localization error: \nNN: {loc_error_nn:.3f} cm\nLM: {loc_error_lm:.3f} cm")

Localization error: 
NN: 1.042 cm
LM: 1.071 cm
